In [1]:
from s3fs.core import S3FileSystem
import turicreate as tc
import os


s3 = S3FileSystem(anon=True)

In [2]:
style_paths = s3.ls("s3://skafos.example.data/StyleTransferImages/styles/")
content_paths = s3.ls("s3://skafos.example.data/StyleTransferImages/content/")

image_paths = style_paths + content_paths

In [13]:
for _dir in ['content', 'styles']:
    if not os.path.exists(_dir):
        os.makedirs(_dir)

In [8]:
styles = tc.SFrame(); content = tc.SFrame();
for p in image_paths:
    _type = p.split("/")[-2]
    local_path = "/".join(p.split("/")[-2:])
    img1 = s3.get("s3://" + p, "./"+local_path)
    if _type == "styles":
        styles = styles.append(tc.load_images(local_path))
    if _type == "content":
        content = content.append(tc.load_images(local_path))
        
    

In [11]:
# Create a Style Transfer model
model = tc.style_transfer.create(styles, content)

Using 'image' in style_dataset as feature column and using 'image' in content_dataset as feature column
Setting max_iterations to be 5000
Using GPU to create model (GeForce GTX 1060)

    You may not have enough free GPU memory to create this model, which may
    result in a fatal crash or lower performance. We recommend that you have at
    least 5548.4 MiB available on each GPU, while currently you have only 4834
    MiB free (out of 6078 MiB) on GPU #0. Please close all other processes that
    may be using this GPU.



KeyboardInterrupt: 

In [ ]:
# Save the model for later use in Turi Create
model.save('./MyStyleTransfer.model')

# Export for use in Core ML
model.export_coreml('./MyStyleTransfer.mlmodel')

In [ ]:
# Load some test images
test_images = tc.load_images('./content/')

# Stylize the test images
stylized_images = model.stylize(test_images)

In [ ]:
stylized_images.explore()

In [ ]:
'''
from common.save_model import *

When you are ready to save your model, your workflow 
should look like the following:

# export to coreml
coreml_model_name = "recommender.mlmodel"
res = model.export_coreml(coreml_model_name)

# compress the model
compressed_model_name, compressed_model = compress_model(coreml_model_name)

# save to Skafos
skafos_save_model(skafos = ska, model_name = compressed_model_name,
                                compressed_model = compressed_model,
                                permissions = 'public')
'''